In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_308104/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 11:01:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 11:01:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/15 11:01:51 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Function to generate random temperature data for a given month and year
def generate_temperature_data(year, month):
    start_date = datetime(year, month, 1)
    end_date = start_date + timedelta(days=pd.Period(start_date, 'M').days_in_month - 1)
    dates = pd.date_range(start=start_date, end=end_date)
    temps = [random.randint(70, 100) for _ in range(len(dates))]
    return pd.DataFrame({'date': dates, 'temp': temps})

# Generate temperature data for each month from January to December for the years 2016 to 2019
data = pd.concat([generate_temperature_data(year, month) for year in range(2016, 2020) for month in range(1, 13)])

# Display the generated data
print(data)




df_person = spark.createDataFrame(data)
df_person.createOrReplaceTempView("high_temps")



         date  temp
0  2016-01-01    72
1  2016-01-02    96
2  2016-01-03    73
3  2016-01-04    77
4  2016-01-05    77
..        ...   ...
26 2019-12-27    85
27 2019-12-28    83
28 2019-12-29    83
29 2019-12-30    98
30 2019-12-31    86

[1461 rows x 2 columns]


In [11]:
query = """
SELECT year(date) year, month(date) month, temp
  FROM high_temps
  WHERE date BETWEEN DATE '2015-01-01' AND DATE '2018-08-31'
"""

In [12]:
spark.sql(query).show()

+----+-----+----+
|year|month|temp|
+----+-----+----+
|2016|    1|  91|
|2016|    1|  73|
|2016|    1|  99|
|2016|    1|  73|
|2016|    1|  83|
|2016|    1|  70|
|2016|    1|  89|
|2016|    1|  80|
|2016|    1|  82|
|2016|    1|  75|
|2016|    1|  96|
|2016|    1|  83|
|2016|    1|  92|
|2016|    1|  87|
|2016|    1|  70|
|2016|    1|  74|
|2016|    1|  75|
|2016|    1|  85|
|2016|    1|  71|
|2016|    1|  81|
+----+-----+----+
only showing top 20 rows



In [15]:
query_1 = """
    SELECT * FROM (
  SELECT year(date) year, month(date) month, temp
  FROM high_temps
  WHERE date BETWEEN DATE '2015-01-01' AND DATE '2018-08-31'
)
PIVOT (
  CAST(avg(temp) AS DECIMAL(4, 1))
  FOR month in (
    1 JAN, 2 FEB, 3 MAR, 4 APR, 5 MAY, 6 JUN,
    7 JUL, 8 AUG, 9 SEP, 10 OCT, 11 NOV, 12 DEC
  )
)
ORDER BY year DESC
"""

In [16]:
spark.sql(query_1).show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+
|year| JAN| FEB| MAR| APR| MAY| JUN| JUL| AUG| SEP| OCT| NOV| DEC|
+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2018|82.5|83.1|85.5|88.8|84.1|83.0|86.4|82.8|NULL|NULL|NULL|NULL|
|2016|82.4|86.1|87.4|83.2|83.2|83.9|85.0|85.4|85.9|85.2|87.1|84.5|
|2017|82.3|85.8|85.6|83.4|84.4|87.0|87.3|86.2|85.4|83.6|86.1|85.1|
+----+----+----+----+----+----+----+----+----+----+----+----+----+



In [4]:
query_2 = """
        SELECT * FROM (
  SELECT year(date) year, month(date) month, temp
  FROM high_temps
  WHERE date BETWEEN DATE '2015-01-01' AND DATE '2018-08-31'
)
"""

In [5]:
spark.sql(query_2).show()

+----+-----+----+
|year|month|temp|
+----+-----+----+
|2016|    1|  72|
|2016|    1|  96|
|2016|    1|  73|
|2016|    1|  77|
|2016|    1|  77|
|2016|    1|  82|
|2016|    1|  79|
|2016|    1| 100|
|2016|    1|  85|
|2016|    1|  90|
|2016|    1|  84|
|2016|    1|  98|
|2016|    1|  87|
|2016|    1|  96|
|2016|    1|  70|
|2016|    1|  90|
|2016|    1|  85|
|2016|    1|  81|
|2016|    1|  99|
|2016|    1|  92|
+----+-----+----+
only showing top 20 rows

